# Week 1 - Introduction to AI

## Train a dog rating network

In this code we will see how to train a dog rating network using the [we rate dogs dataset](https://www.kaggle.com/datasets/terencebroad/we-rate-dogs-images-ratings-and-captions).

The code has been written for you. Your job today is to run the code to train a dog rating network and to write comments for the code. Work your way through the code and try to write a comment for each line. Lines that you do not understand and cannot write comments for, add them to the miro board (link in moodle) for dicussion at the end of the session.

First lets do some imports:

In [6]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

Import some more functions:

In [7]:
# Import dog rating model from src/model.py
from src.model import DogRatingNetwork
# Import We rate dogs dataset loading class from src/data_util.py
from src.data_util import WeRateDogsDataset
# Import data loader class from: https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
from torch.utils.data import DataLoader

Here we will define some hyperparameters. In your comment can you describe what each variable is for? (It's ok if you don't know everything!)

In [8]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyperparameter coniguring how much rolling average over gradients there is
momentum = 0.9
# Number of cycles through the dataset in training
num_epochs = 10
# The number of data samples processed at once in training
batch_size = 100
# Hyperparameter that controls how aggressive weight updates are in training
learn_rate = 0.001
# Path to training dataset
data_path = '../data/class-datasets/we-rate-dogs-dataset/'

Here we define image transformations. Can you describe what each transform is doing?

In [9]:
# The set of transforms used when loading in images
transform = transforms.Compose(
    [   
        # This transform resizes images so that the shortest length (width or height) is 64 pixels
        torchvision.transforms.Resize(64, antialias=True),
        # Crop image to a 64x64 pixel square
        torchvision.transforms.CenterCrop(64),
        # Convert to pytorch tensor
        transforms.ToTensor(),
        # Normalise pixel values to range 0-1, centered around 0.5
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

Defining our dataset and dataloader:

In [11]:
# Create dataset class 
# This defines where the dataset is and how data is loaded into the code
dataset = WeRateDogsDataset(data_path, transform=transform)
# Create dataloader class 
# This controls how batches are loaded in sequence
# Here we are randomly shuffling the data each time we load an epoch
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

Initialising core objects for training. What are all of these objects?

Try referring to the [PyTorch reference](https://pytorch.org/docs/stable/index.html) for functions or objects that you are unsure of. 

In [12]:
# Create our dog rating neural network
model = DogRatingNetwork()
# Move the weights of our model to our specified hardware device
model.to(device)
# Loss function for training
# This is the mean squared error loss 
criterion = nn.MSELoss()
# Stochastic gradient descent optimiser for updating weights of model
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

This is the training loop. Again, can you describe what each line of code does?

In [ ]:
# Variable for running loss
# Start with a very high number so that we can track the lowest running value
best_loss = 100000

# For loop iterating over the number of epochs set in num_epochs
for epoch in range(num_epochs): 
    
    # Create a variable for the running loss over the whole epoch
    running_loss = 0.0
    
    # For loop iterating over all the batches in one cycle of the dataset
    for i, data in enumerate(train_loader, 0):
        # Extract image batch and dog sore labels from the tuple 'data'
        inputs, labels = data
        # Move images to hardware device tensor
        inputs = inputs.to(device)
        # Move labels to hardware device tensor
        labels = labels.to(device)
        # Clear gradients from previous iterations
        optimizer.zero_grad()
        # Process images to get predictions
        outputs = model(inputs)
        # Compare predictions to the true labels of dog scores and get loss
        loss = criterion(outputs, labels.unsqueeze(1))
        # Backpropagate loss through the network
        loss.backward()
        # Update the weights of the network
        optimizer.step()
        # Add loss for this iteration to running losss
        running_loss += loss.item()
    
    # Track output after single epoch
    print(f'Epoch {epoch + 1}, total loss: {running_loss:.3f}')
    # If the current loss is better than the best overall loss
    if running_loss < best_loss:
        # Update best overaall to current overall loss
        best_loss = running_loss
        # Save model outputs
        torch.save(model.state_dict(), 'model.pt')
    
    # Reset the value of the running loss before starting new epoch
    running_loss = 0.0

Once you have tried to write comments for all the code here. Move on to the file test-dog-rating-network.ipynb to test out your trained network. 

If you want a further challenge, look at the files in the `src` folder (`src/data_util.py`, (`src/model.py`). Can you write comments for the code in there?